# File mover
This notebook is intended to query the MYSQL database for a list of files and copy them to a separate folder using multithreading. The main idea for this is to reduce the need for RDP when developing code or latency by not having to FTP to the home-server.

In [6]:
import pandas as pd
import sys
sys.path.append('../utils')
import config_handling as conf
from database import Database
from file_io import mt_copier


In [2]:
# Connect to database
config = conf.read_config('../config/automotive.conf.ini')
config.read('config.ini')
connection_type = config['settings']['connection']
connection_type
user = config[connection_type]['user']
pw = config[connection_type]['pw']
host = config[connection_type]['host']
db = config[connection_type]['db']
port = config[connection_type].getint('port')
db = Database(host,
              port,
              user,
              pw,
              db
              )
db.connect()
db.start_transaction()
#image directory: 
basedir = config['settings']['image_directory']

Connection established


In [3]:
#get all file_paths for cars listed in Belgium: 

query = """SELECT image_path FROM images 
    JOIN listings on listings.id = images.listing_id
    where listings.countrycode = 'B';
"""
results = db.execute_query(query)

In [4]:
image_paths = [item['image_path'] for item in results]
root_dest = input('What is the root directory of the destination files?')

In [7]:
mt_copier(basedir, image_paths, root_dest, max_workers=10)

  0%|          | 0/1353280 [00:00<?, ?it/s]

100%|██████████| 1353280/1353280 [01:20<00:00, 16860.19it/s]
